In [6]:
from google.cloud import api_keys_v2
from google.cloud.api_keys_v2 import Key

def create_api_key(project_id: str, suffix: str) -> Key:
    """
    Creates and restrict an API key. Add the suffix for uniqueness.
    (Developer):
    1. Before running this sample,
      set up ADC as described in https://cloud.google.com/docs/authentication/external/set-up-adc
    2. Make sure you have the necessary permission to create API keys.
    Args:
        project_id: Google Cloud project id.
    Returns:
        response: Returns the created API Key.
    """
    # Create the API Keys client.
    client = api_keys_v2.ApiKeysClient()

    key = api_keys_v2.Key()
    key.display_name = f"API key - {suffix}"

    # Initialize request and set arguments.
    request = api_keys_v2.CreateKeyRequest()
    request.parent = f"projects/{project_id}/locations/global"
    request.key = key

    # Make the request and wait for the operation to complete.
    response = client.create_key(request=request).result()

    print(f"Successfully created an API key: {response.name}")
    # For authenticating with the API key, use the value in "response.key_string".
    # To restrict the usage of this API key, use the value in "response.name".
    return response


In [35]:
import vertexai
from vertexai.language_models import TextGenerationModel, ChatModel, InputOutputTextPair, CodeChatModel

In [29]:
# LLM for general purposes
def google_llm(inquiry):
    vertexai.init(project='project_name', location='us-central1')
    # TODO developer - override these parameters as needed:
    parameters = {
        "temperature": 0.9,  # Temperature controls the degree of randomness in token selection.
        "max_output_tokens": 1024,  # Token limit determines the maximum amount of text output.
        "top_p": 0.8,  # Tokens are selected from most probable to least until the sum of their probabilities equals the top_p value.
        "top_k": 40,  # A top_k of 1 means the selected token is the most probable among all tokens.
    }

    model = TextGenerationModel.from_pretrained("text-bison@001")
    response = model.predict(
        inquiry,
        **parameters,
    )

    return response.text

In [ ]:
# LLM for call center
def google_chat(message, temperature: float = 0.2):
    chat_model = ChatModel.from_pretrained("chat-bison@001")

    # TODO developer - override these parameters as needed:
    parameters = {
        "temperature": temperature,  # Temperature controls the degree of randomness in token selection.
        "max_output_tokens": 256,  # Token limit determines the maximum amount of text output.
        "top_p": 0.95,  # Tokens are selected from most probable to least until the sum of their probabilities equals the top_p value.
        "top_k": 40,  # A top_k of 1 means the selected token is the most probable among all tokens.
    }

    chat = chat_model.start_chat(
        context="My name is Bot. You are a call center supervisor, knowledgable of the best practices.",
        examples=[
            InputOutputTextPair(
                input_text="What are the main mistakes when answering a call?",
                output_text="Not start by introducing yourself and voicing your interest in solving the clients needs",
            ),
        ],
    )

    response = chat.send_message(
        message, **parameters
    )

    return response.text

In [32]:
# LLM for code
def google_code(inquiry):
    vertexai.init(project="project_name", location="us-central1")
    chat_model = CodeChatModel.from_pretrained("codechat-bison")
    parameters = {
        "candidate_count": 1,
        "max_output_tokens": 1024,
        "temperature": 0.9
    }
    chat = chat_model.start_chat()
    response = chat.send_message(
        inquiry, 
        **parameters
    )
    
    return response.text